In [25]:
from multiprocessing import Process, Queue

In [4]:
import asyncio

In [48]:
async def tcp_authenticate_request(S_IP, S_PORT, q):
    """ TCP Client to connect to RMC Server """

    q.put(f"... Connecting to server {S_IP}:{S_PORT}")
    q.put(['a','b','c'])
    try:
        reader, writer = await asyncio.open_connection(S_IP, S_PORT)
    except ConnectionRefusedError:
        q.put(f"!!! Connection to server {S_IP}:{S_PORT} refused")
        return
    except TimeoutError:
        q.put(f"!!! Connection to server {S_IP}:{S_PORT} timed out")
        return

    q.put("... Sent request")
    q.put('>>> Sent encrypted authentication request')
    writer.write(encrypted)

    auth_reply = await reader.read(1024)

    try:
        auth_reply = Fernet(SPASS).decrypt(auth_reply)
        auth_reply = auth_reply.decode()
        q.put(f">>> Received Authentication Reply: {auth_reply}")

        # TODO: Add IP_SUBNET
        PKEYS, IP_ASSIGNED = auth_reply.split(',')

    except InvalidToken as e:
        q.put(f"... Received Authentication Error: {auth_reply.decode()}")
        q.put(f"Authentication Error: {auth_reply.decode()}")
        pass

    writer.close()

def tcp_client(q):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(tcp_authenticate_request("127.0.0.1", 1234, q))
    loop.close()
    
    q.put("DONE")

In [57]:
q = Queue()
p = Process(target=async_tcp_client, args=(q,))
p.start()

In [58]:
while not q.empty():
    got = q.get_nowait()
    print(got, type(got))

... Connecting to server 127.0.0.1:1234 <class 'str'>
['a', 'b', 'c'] <class 'list'>
!!! Connection to server 127.0.0.1:1234 refused <class 'str'>
DONE <class 'str'>


In [30]:
p.kill()

In [31]:
p

<Process(Process-9, stopped)>

In [41]:
?Queue

Signature: Queue(maxsize=0)
Docstring: Returns a queue object
File:      ~/opt/anaconda3/lib/python3.7/multiprocessing/context.py
Type:      method
